In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Conv2D, Conv2DTranspose, InputLayer
from tensorflow.keras.models import Sequential
import os

In [7]:
def fsrcnn():
    model = Sequential()
    model.add(InputLayer(input_shape=(270, 480, 3)))
    model.add(Conv2D(56, 5, padding='same', activation='relu'))
    model.add(Conv2D(12, 1, padding='same', activation='relu'))
    for i in range(4):
        model.add(Conv2D(12, 3, padding='same', activation='relu'))
    model.add(Conv2DTranspose(3, 9, strides=4, padding='same',))
    model.compile(optimizer=tf.optimizers.Adam(1e-1), loss=tf.losses.mse, metrics=['mse'])
    return model

model = fsrcnn()
plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', verbose=0, mode='min', factor=0.1, patience=6)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', verbose=0, mode='min', patience=25)
checkpoint = keras.callbacks.ModelCheckpoint('srcnn.h5', monitor='val_loss', verbose=0, mode='min', save_best_only=True)

In [39]:
X = []
y = []
for i in range(1, 51):
    img_path = os.path.join('youku', 'IMG_LOW', 'Youku_00000_l', '%03d.bmp' % i)
    img_l = cv2.imread(img_path)
    img_path = os.path.join('youku', 'IMG_HIGH', 'Youku_00000_h_GT', '%03d.bmp' % i)
    img_h = cv2.imread(img_path)
    X.append(img_l)
    y.append(img_h)

In [41]:
X = np.array(X)
y = np.array(y)

In [43]:
model.fit(X, y, epochs=10, batch_size=2, verbose=1, shuffle=True, validation_data=(X, y), callbacks=[plateau, early_stopping, checkpoint])
model.evaluate(X, y, verbose=0)
pic_super = model.predict(X, verbose=0, batch_size=1)
cv2.imwrite("./fsrcnn_00.bmp", pic_super[0])

25/25 [==============================] - 33s 1s/step - loss: 26665.9824 - mse: 26665.9824 - val_loss: 26666.0000 - val_mse: 26665.9922 - lr: 0.0100
Epoch 4/10
25/25 [==============================] - 33s 1s/step - loss: 26665.9941 - mse: 26665.9941 - val_loss: 26665.9922 - val_mse: 26665.9922 - lr: 0.0100

Epoch 1/10
25/25 [==============================] - 32s 1s/step - loss: 26665.9668 - mse: 26665.9688 - val_loss: 26665.9668 - val_mse: 26665.9688 - lr: 1.0000e-03

True

In [46]:
def espcn():
    inputs = keras.layers.Input(shape=(270, 480,3))
    cnn = keras.layers.Conv2D(64, 5, padding='same', activation='relu')(inputs)
    cnn = keras.layers.Conv2D(32, 3, padding='same', activation='relu')(cnn)
    cnn = keras.layers.Conv2D(3 * 4 ** 2, 3, padding='same')(cnn)
    cnn = tf.reshape(cnn, [-1, 270, 480, 4, 4, 3])
    cnn = tf.transpose(cnn, perm=[0,1,2,3,4,5])
    outputs=tf.reshape(cnn, [-1, 270 * 4, 480 * 4, 3])
    model = keras.models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=tf.optimizers.Adam(1e-1), loss=tf.losses.mse, metrics=['mse'])

    return model

model = espcn()
plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', verbose=0, mode='min', factor=0.1, patience=6)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', verbose=0, mode='min', patience=25)
checkpoint = keras.callbacks.ModelCheckpoint('srcnn.h5', monitor='val_loss', verbose=0, mode='min', save_best_only=True)

model.fit(X, y, epochs=10, batch_size=2, verbose=1, shuffle=True, validation_data=(X, y), callbacks=[plateau, early_stopping, checkpoint])
model.evaluate(X, y, verbose=0)
pic_super = model.predict(X, verbose=0, batch_size=1)
cv2.imwrite("./espcn.bmp", pic_super[0])




Epoch 1/10
25/25 [==============================] - 33s 1s/step - loss: 1602094976.0000 - mse: 1602094848.0000 - val_loss: 26711.5977 - val_mse: 26711.5996 - lr: 0.1000

True